In [45]:
import pandas as pd
import keras as k
import numpy as np
from keras.preprocessing.text import Tokenizer
# from utils import get_data, generate_output, guess_human, seed_sequence, get_embeddings, find_closest

In [52]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Masking, Embedding

In [12]:
data = pd.read_csv('./dataset/dataset.csv')

In [13]:
data.head()

,patent_abstract,patent_title
0,""" This invention is a novel high-speed neural ...","""Electronic neural network for solving """"trave..."
1,Systems and methods are disclosed to recognize...,3D convolutional neural networks for automatic...
2,A supervised procedure for obtaining weight va...,Accelerated training apparatus for back propag...
3,A method of accelerating the training of an ar...,Accelerating learning in neural networks
4,An apparatus is described herein. The apparatu...,Accumulator constrained quantization of convol...


In [14]:
# training_dict, word_idx, idx_word, sequences = get_data('./dataset/dataset.csv', training_len = 50)

In [15]:
data.columns

Index(['patent_abstract', 'patent_title'], dtype='object')

In [17]:
abstracts = data['patent_abstract'].values

In [37]:
abstracts[100][:300]

'Methods and apparatuses for backlash compensation. A dynamics inversion compensation scheme is designed for control of nonlinear discrete-time systems with input backlash. The techniques of this disclosure extend the dynamic inversion technique to discrete-time systems by using a filtered prediction'

In [29]:
tokenizer = Tokenizer(num_words = None, filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n', lower = False, split = ' ')
tokenizer.fit_on_texts(abstracts)
sequences = tokenizer.texts_to_sequences(abstracts)

In [32]:
sequences[100][:15]

[885, 4, 3034, 7, 3035, 6565, 17, 1287, 2604, 2605, 977, 8, 658, 7, 51]

In [43]:
# mapping indexes to words using index words
idx_word = tokenizer.index_word

' '.join(idx_word[w] for w in sequences[100][:42])

'Methods and apparatuses for backlash compensation. A dynamics inversion compensation scheme is designed for control of nonlinear discrete time systems with input backlash. The techniques of this disclosure extend the dynamic inversion technique to discrete time systems by using a filtered prediction,'

In [46]:
features = []
labels = []

training_length = 50

for seq in sequences:
#     create multiple training examples form each sequence
    for i in range(training_length, len(seq)):
        extract = seq[i - training_length:i + 1]
        features.append(extract[:-1])
        labels.append(extract[-1])
        
features = np.array(features)

In [48]:
len(features)

84154

In [51]:
# One hot encoding the labels
num_words = len(idx_word) + 1
label_array = np.zeros((len(features), num_words), dtype = np.int8)

for example_index, word_index in enumerate(labels):
    label_array[example_index, word_index] = 1
    
label_array.shape

(84154, 10132)

In [57]:
model = Sequential()
model.add(Embedding(
    input_dim=num_words,
    input_length = training_length,
    output_dim=100,
    weights=[embedding_matrix],
    trainable=False,
    mask_zero=True))

model.add(Masking(mask_value=0.0))

model.add(LSTM(
    64, 
    return_sequences=False, 
    dropout=0.1, 
    recurrent_dropout=0.1))

model.add(Dense(64, activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(num_words, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

AttributeError: 'NoneType' object has no attribute 'shape'